### Short technical report on models' performance and training

__Introduction__<br>
This short notebook provides comprehensive information on the process of creating three Transformer models specifically designed for German-English machine translation. Throughout this notebook, you will find explanations of the steps involved in cleaning and preprocessing the data, as well as converting it into suitable datasets for training the models. Additionally, this notebook offers a concise yet insightful overview of the training process, covering models with different sizes and architectures.

__Table of contents__<br>
1. Dataset
2. Models
3. Training
4. Evaluation
5. Conclusion
<br>

<a id=1></a>
__1. Dataset__<br>
The dataset utilized in this project is the WMT 14 English-German Dataset, which comprises 4.6 million sentence pairs in Text file format. Prior to its usage, the data underwent a meticulous cleaning and standardization process. This process involved several steps, including converting the text to lowercase, eliminating trailing punctuation symbols, replacing unusual symbols, and removing incomplete parentheses. Furthermore, shorter sentences were padded to a length of 100 tokens, and special tokens such as \<SOS> (start of sequence) and \<EOS> (end of sequence) were appended to the target sequences. To ensure high-quality data, sentences were filtered based on the following four rules: 

1. Sentences shorter than 10 and longer than 100 tokens were removed from the dataset. 
2. Sentences containing a proportion of \<UNK> tokens greater than 15% of the total tokens were eliminated. 
3. Sentences were discarded if one of them was at least two times longer than the other. 
4. Digits in each sentence were separated with whitespace (eg. 123 -> 1 2 3). 

By applying these filtering rules and preprocessing techniques, the dataset was refined to ensure its suitability for training robust German-English machine translation models. Below charts show distribution of lengths in both English and German datasets as well as distribution of \<UNK> tokens. Total vocabulary size for both English and German datasets was 10,000 as it was enough to ensure that total number of \<UNK> tokens was lower that 0.5%. Statistics on \<UNK> tokens can bee seen in figure 1.<br><br>

![Distribution of sentence lengths](../plots/vocab-10000/en-de-dist-10000.png)<br>
Figure 1.1: Distribution of sentence lengths.<br><br>

![Distribution of sentence lengths in log scale](../plots/vocab-10000/en-de-dist-10000-log.png)<br>
Figure 1.2: Distribution of sentence lengths in log scale.<br><br>

![Distribution of sentence lengths](../plots/vocab-10000/en-de-unk-10000-log.png)<br>
Figure 1.3: Distribution of \<UNK> tokens in sentences in log scale.<br><br>

![Distribution of sentence lengths in log scale](../plots/vocab-10000/en-de-unk-perc-10000-log.png)<br>
Figure 1.4: Distribution of percentages of \<UNK> tokens in sentences in log scale.<br><br>


After the completion of preprocessing, cleaning, and tokenization steps, the data was divided into three distinct datasets: *train*, *dev*, and *test*. These datasets were allocated proportions of 70%, 15%, and 15%, respectively. The *train* set encompassed approximately 2.8 million examples, while both the *dev* and *test* sets contained approximately 0.6 million examples each. Upon further analysis, it was determined that the total number of tokens in the dataset (excluding padding and special tokens) amounted to 171 million tokens. Among these, the German text accounted for 88 million tokens, while the English text consisted of 83 million tokens. This information provides valuable insights into the distribution of tokens between the two languages. Moreover, it is noteworthy to mention the average word-to-token ratio for each language. For German, the average ratio was 0.71, indicating that there were, on average, 0.71 words per token. Similarly, for English, the average ratio was 0.91, implying that there were, on average, 0.91 words per token. These ratios serve as an indicator of the average length and complexity of the sentences in the respective languages.
<br><br>

<a id=2></a>
__2. Models__<br>
Due to limited computing resources, a total of three models were trained for this project. The first model, referred to as trs-18m, was based on the PyTorch implementation of the Transformer architecture and contained approximately 18 million parameters. The remaining two models, xfr-44m and xfr-18m, were built using the xFormers library and had 44 million and 18 million parameters, respectively. To eliminate the learning rate warmup, a LayerNorm layer was introduced before the MultiHeadAttention and MLP components in all three models. This modification aimed to enhance the training process and improve the models' performance. However, there were minor architectural differences and variations in the training processes among the models. For the xFormer models, Weight Decay (WD) was not applied to biases and LayerNorm layers, distinguishing them from the PyTorch implementation. Additionally, a LayerNorm layer was added before the classification head, further refining the model's output. One notable distinction in the xFormer models was the utilization of rotary embeddings instead of regular embeddings found in the PyTorch implementation. For a more comprehensive understanding of each model's architecture and please refer to Table 1. <br>

|  Model  | Params | Seq. Len. | Vocab Size | Emb. Dim. | Num. Heads | Ffwd. Dim. |  Act. | Num. Layers |
|---------|--------|-----------|------------|-----------|------------|------------|-------|-------------|
| trs-18m | 18M    | 100       | 10,000     | 256       | 4          | 1024       | gelu  | 6           |
| xfr-18m | 18M    | 100       | 10,000     | 256       | 4          | 1024       | gelu  | 6           |
| xfr-44m | 44M    | 100       | 10,000     | 384       | 6          | 1536       | gelu  | 8           |


Table 1. Architectures of trained models.
<br><br>

<a id=3></a>
__3. Training__<br>
Training for all models was carried out over 5 epochs utilizing Google's T4 GPU with float-16 precision in order to increase speed. A relatively small batch size of 128 was maintained throughout the training process due to the observed generalization gap that occurred with larger batch sizes. To optimize the learning process, a cosine decay strategy was employed, gradually reducing the learning rate towards 0 as the training progressed. This approach helps to fine-tune the model and achieve better convergence. All models were trained using AdamW optimizer (betas: 0.9, 0.999). For more comprehensive information regarding the training process, please refer to Table 2. <br>

| Model    | Train Loss | Dev Loss | Init. LR | Batch Size | Dropout | Weight Decay | Betas      | Train Time |
|----------|------------|----------|----------|------------|---------|--------------|------------|------------|
| trs-18m  | 0.6483     | 0.5983   | 3e-3     | 128        | 0.1     | 0.1          | 0.9, 0.999 | ~17h       |
| xfr-18m  | 0.6309     | 0.6159   | 5e-3     | 128        | 0.1     | 0.1          | 0.9, 0.999 | ~15h       | 
| xfr-44m  | 0.6199     | 0.5762   | 5e-3     | 128        | 0.05    | 0.1          | 0.9, 0.999 | ~18.5h     |


Table 2. Models' losses after 5th epoch. <br><br>

The discrepancy in training time between PyTorch and xFormer models can be attributed to the more efficient attention implementation in xFormers. This optimized attention mechanism leads to a reduction in training speed compared to PyTorch.
<br><br>

<a id=4></a>
__4. Evaluation__<br>
Despite evaluating the regular loss, the models' performance was assessed using three additional metrics: Perplexity, BLEU, and ROUGE. However, due to the large size of the training set, it was not feasible to test the models on the entire dataset within a reasonable timeframe (testing would take approximately 22 hours on a GTX 1660 GPU). As a result, a random sample of 1000 instances was drawn from the test set for evaluation purposes. Table 3 displays the scores achieved by each of the models on the selected random sample. <br>

| Model    | Perplex. | BLEU  | ROUGE-1 | ROUGE-2 | ROUGE-L |
|----------|----------|-------|---------|---------|---------|
| trs-18m  | 1.63     | 33.50 | 0.48    | 0.27    | 0.44    | 
| xfr-18m  | 1.86     | 41.72 | 0.47    | 0.25    | 0.43    |
| xfr-44m  | 1.69     | 34.81 | 0.48    | 0.26    | 0.44    |

Table 3. Comparison of model performance based on different metrics. <br><br>

Despite xfr-44m having lower loss and perplexity compared to xfr-18m, it exhibits a significantly lower BLEU score. This observation suggests that even with the increased number of parameters (44 million), xfr-44m may require more training data or additional training time to fully leverage its potential. On the other hand, the PyTorch trs-18m model faces difficulties in translating shorter sequences. It tends to repeat the last word multiple times unless the input sentence ends with a "." sign. This issue indicates a limitation in the model's ability to handle shorter sentences effectively. After thorough evaluation and testing, it has been determined that xfr-18m outperforms both xfr-44m and trs-18m in terms of efficiency and translation quality. As a result, the results obtained from xfr-18m will be presented.

Below are 10 sentences generated by ChatGPT with increasing complexity and length to assess the performance of the model. Model is not case sensitive, so the sentences are capitalized.

1. **Sentence**: Das Buch ist rot.<br>**Translation**: The book is red.<br>**Reference**: The book is red.<br><br>
2. **Sentence**: Ich trinke Kaffee.<br>**Translation**: I drink coffee.<br>**Reference**: I drink coffee.<br><br>
3. **Sentence**: Wo ist der Bahnhof?<br>**Translation**: Where is the train station? <br>**Reference**: Where is the train station?<br><br>
4. **Sentence**: Der Hund rennt im Park.<br>**Translation**: The dog races in the park.<br>**Reference**: The dog is running in the park.<br><br>
5. **Sentence**: Meine Familie und ich werden morgen in den Urlaub fahren.<br>**Translation**: My family and I will go to the holiday tomorrow.<br>**Reference**: My family and I are going on vacation tomorrow.<br><br>
6. **Sentence**: In meiner Freizeit spiele ich gerne Gitarre und lese Bücher.<br>**Translation**: In my leisure time I like guitar and reading books.<br>**Reference**: In my free time, I enjoy playing the guitar and reading books.<br><br>
7. **Sentence**: Letzte Woche habe ich meine Prüfungen bestanden und bin sehr stolz auf mich.<br>**Translation**: Last week I finished my exams and I am very proud.<br>**Reference**: Last week, I passed my exams and I am very proud of myself.<br><br>
8. **Sentence**: Der Klimawandel ist eine der größten Herausforderungen unserer Zeit und erfordert dringende Maßnahmen.<br>**Translation**: Climate change is one of the greatest challenges of our time and requires urgent action. <br>**Reference**: Climate change is one of the biggest challenges of our time and requires urgent actions.<br><br>
9. **Sentence**: Obwohl ich sehr müde bin, werde ich die ganze Nacht durcharbeiten, um meine Deadline einzuhalten.<br>**Translation**: Although I am very tired, I will work all night, to keep my deadline up.<br>**Reference**: Even though I am very tired, I will work all night to meet my deadline.<br><br>
10. **Sentence**: Als ich gestern in die Stadt ging, traf ich zufällig einen alten Freund, den ich seit Jahren nicht gesehen hatte. Wir hatten eine lange und angeregte Unterhaltung über alte Zeiten.<br>**Translation**: When I went to the city yesterday, I accidentally met an old friend, who had not seen me for years. we had a long and excited conversation about old times.<br>**Reference**: Yesterday, when I went into town, I randomly ran into an old friend whom I hadn't seen in years. We had a long and lively conversation about old times.<br>

One downside of the model is that it is not case-sensitive, so manually capitalizing the first letter of each sentence is necessary. Additionally, the model does not automatically capitalize names or cities. Another limitation is that a significant portion of the training data ended with a "." sign, resulting in the model automatically adding a "." at the end of each translated sentence. Interestingly, the model occasionally encounters difficulties in translating short and simple sentences, but performs well on longer and more complex ones. This could be attributed to the fact that sentences shorter than 10 words were removed from the training dataset, leading to a lack of exposure to very short sentence structures. It is worth noting that these downsides are inherent to the model's training data and architecture. Despite these limitations, the model demonstrates satisfactory performance on longer and more challenging sentences, highlighting its ability to handle complex linguistic structures effectively. The model can also translate sentences that contain numbers, dates etc. However, the results are mixed. Below a few examples of such ability is presented.

1. **Sentence**: Ich habe am 10.10.21 meinen Geburtstag.<br>**Translation**: I have a birthday on 10.10. 21.<br>**Reference**: I have my birthday on the 10th of October 2021.<br><br>
2. **Sentence**: Wir werden uns am 5. März 2023 treffen.<br>**Translation**: We will meet on 5 march 2023.<br>**Reference**: We will meet on the 5th of March 2023.<br><br>
3. **Sentence**: Ich habe den Vertrag am 30.06. unterschrieben.<br>**Translation**: I signed the treaty on 30. 06.<br>**Reference**: I signed the contract on the 30th of June.<br><br>
4. **Sentence**: Unser Flug geht um 8:30 Uhr morgens.<br>**Translation**: Our flight is about 8:30 a.m.<br>**Reference**: Our flight departs at 8:30 a.m.<br><br>
5. **Sentence**: Das Hotelzimmer kostet 120 Euro pro Nacht.<br>**Translation**: The hotel room costs 120 euros per night.<br>**Reference**: The hotel room costs 120 euros per night.<br>

Lastly, I asked to ChatGPT to generate lengthy sentences encompassing a plethora of specific and intricate terminologies. Here are the outcomes presented below.

1. **Sentence**: Die komplexe geopolitische Lage in der heutigen Welt erfordert eine ausgewogene Außenpolitik, die auf diplomatischen Dialog, Zusammenarbeit und dem Streben nach Konfliktlösungen basiert.<br>**Translation**: The complex geopolitical situation in today s world requires balanced foreign policy based on diplomatic dialogue, cooperation and conflict resolution.<br>**Reference**: The complex geopolitical situation in today's world requires a balanced foreign policy based on diplomatic dialogue, cooperation, and the pursuit of conflict resolution.<br><br>
2. **Sentence**: Die stetige Zunahme von Cyberkriminalität und digitalen Bedrohungen erfordert eine verstärkte Zusammenarbeit zwischen Regierungen, Unternehmen und der Gesellschaft, um effektive Sicherheitsmaßnahmen und robuste Verteidigungssysteme zu entwickeln.<br>**Translation**: The constant increase in cyber crime and digital threats requires enhanced cooperation between governments, companies and society, and effective security measures and robust defence systems.<br>**Reference**: The steady increase in cybercrime and digital threats calls for enhanced collaboration between governments, businesses, and society to develop effective security measures and robust defense systems.<br><br>
3. **Sentence**: Die zunehmende soziale Ungleichheit und die Herausforderungen im Bildungssystem erfordern eine ganzheitliche Herangehensweise, um gleiche Chancen für alle Bevölkerungsgruppen zu gewährleisten und den Zugang zu qualitativ hochwertiger Bildung zu verbessern.<br>**Translation**: The increasing social inequality and the challenges of education systems require a holistic approach, in order to ensure equal opportunities for all population groups and to improve access to high quality education.<br>**Reference**: The growing social inequality and challenges in the education system require a holistic approach to ensure equal opportunities for all population groups and improve access to quality education.<br><br>
4. **Sentence**: Der technologische Fortschritt hat eine tiefgreifende Transformation der Arbeitswelt bewirkt, was die Notwendigkeit einer kontinuierlichen Weiterbildung und Anpassungsfähigkeit der Arbeitnehmer hervorhebt, um den Anforderungen einer sich wandelnden Arbeitsumgebung gerecht zu werden.<br>**Translation**: Technological progress has resulted in a profound transformation of the working world, which highlights the need for continuous training and adaptability of workers, in order to meet the requirements of changing working environment.<br>**Reference**: Technological advancements have brought about a profound transformation of the labor market, highlighting the need for continuous upskilling and adaptability of workers to meet the demands of a changing work environment.<br><br>
5. **Sentence**: Die effektive Bewältigung globaler Gesundheitskrisen erfordert eine enge Zusammenarbeit zwischen Regierungen, internationalen Organisationen, medizinischen Fachkräften und der Zivilgesellschaft, um Präventionsmaßnahmen zu stärken, den Zugang zu Gesundheitsversorgung zu verbessern und die Pandemievorsorge zu stärken.<br>**Translation**: Effective management of global health crises requires close cooperation between governments, international organisations, medical professionals and civil society, to strengthen prevention measures, to improve access to health care and to strengthen pandemic prevention.<br>**Reference**: The effective management of global health crises requires close collaboration between governments, international organizations, medical professionals, and civil society to strengthen preventive measures, improve access to healthcare, and enhance pandemic preparedness.<br><br>

<a id=5></a>
__5. Conclusion__<br>
All things considered, the quality of translations achieved by the model is commendable, especially when considering that it was trained from scratch utilizing freely available Google resources. While the model may encounter challenges with shorter sequences, it demonstrates proficiency in handling longer ones. In terms of future work, it is crucial to conduct further evaluations to assess the model's performance on sentences containing rare or less frequent words. This evaluation would provide valuable insights into how the model copes with translating uncommon vocabulary. Additionally, visualizing the attention mechanism employed by the model would greatly contribute to our understanding of how it attends to different positions within the input sentence. This visualization would shed light on the model's decision-making process and the relative importance it assigns to various words or phrases during translation.